In [1]:
from pssmgen import PSSM

In [2]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [3]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [24]:
modification_site = pd.read_csv("/home/yuan/Documents/PTM_N_link.csv")

In [25]:
modification_site.iloc[1:5,]

,Uniprot,PDB,CHAIN,uniprot_site,pdb_site,pdb_modres,dbptm_annot,PTM,scop_class,aa_uniprot,organism
1,O00206,2z62,A,173,173,GLYCOSYLATION SITE,N-linked (GlcNAc...).,N-linked Glycosylation,None,N,HUMAN (Human)
2,O00206,2z62,A,205,205,GLYCOSYLATION SITE,N-linked (GlcNAc...).,N-linked Glycosylation,None,N,HUMAN (Human)
3,O00206,2z63,A,309,309,GLYCOSYLATION SITE,N-linked (GlcNAc...).,N-linked Glycosylation,None,N,HUMAN (Human)
4,O00206,2z63,A,497,497,GLYCOSYLATION SITE,N-linked (GlcNAc...).,N-linked Glycosylation,None,N,HUMAN (Human)


In [26]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"pdb_site"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"pdb_site"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"pdb_site"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])

In [27]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()
unique_list = unique_list.reset_index(drop=True)

In [28]:
unique_list.head()

,PDB,CHAIN
0,2z62,A
1,2z63,A
2,2z65,A
3,2z65,B
4,2z66,A


In [29]:
cd_hit_7 = pd.read_csv("/home/yuan/Documents/cdhit0.7_N_link.csv")

In [30]:
cd_hit_7.columns = ["PDB","CHAIN"]
cd_hit_7.head()

,PDB,CHAIN
0,1a6a,C
1,1a7c,A
2,1a7s,A
3,1apy,C
4,1apy,D


In [31]:
for index,oneRow in cd_hit_7.iterrows():
    PDB = PDBList()
    pdbID = oneRow["PDB"]
    chainOrder = oneRow["CHAIN"]
    try:
        retrieve_status = PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "/mnt/HD1/pdb_data/" + pdbID+"/pdb", file_format="pdb")
        os.rename(retrieve_status,retrieve_status[0:(len(retrieve_status)-3)]+'pdb')
    except:
        print("bad chain")
    print(retrieve_status)

/mnt/HD1/pdb_data/1a6a/pdb/pdb1a6a.ent
/mnt/HD1/pdb_data/1a7c/pdb/pdb1a7c.ent
/mnt/HD1/pdb_data/1a7s/pdb/pdb1a7s.ent
/mnt/HD1/pdb_data/1apy/pdb/pdb1apy.ent
/mnt/HD1/pdb_data/1apy/pdb/pdb1apy.ent
/mnt/HD1/pdb_data/1au1/pdb/pdb1au1.ent
/mnt/HD1/pdb_data/1auk/pdb/pdb1auk.ent
/mnt/HD1/pdb_data/1b0f/pdb/pdb1b0f.ent
/mnt/HD1/pdb_data/1b41/pdb/pdb1b41.ent
/mnt/HD1/pdb_data/1bhg/pdb/pdb1bhg.ent
/mnt/HD1/pdb_data/1bik/pdb/pdb1bik.ent
/mnt/HD1/pdb_data/1bmo/pdb/pdb1bmo.ent
/mnt/HD1/pdb_data/1bsi/pdb/pdb1bsi.ent
/mnt/HD1/pdb_data/1by2/pdb/pdb1by2.ent
/mnt/HD1/pdb_data/1c1z/pdb/pdb1c1z.ent
/mnt/HD1/pdb_data/1ccz/pdb/pdb1ccz.ent
/mnt/HD1/pdb_data/1de4/pdb/pdb1de4.ent
/mnt/HD1/pdb_data/1dfv/pdb/pdb1dfv.ent
/mnt/HD1/pdb_data/1dlh/pdb/pdb1dlh.ent
/mnt/HD1/pdb_data/1dmt/pdb/pdb1dmt.ent
/mnt/HD1/pdb_data/1dr9/pdb/pdb1dr9.ent
/mnt/HD1/pdb_data/1elv/pdb/pdb1elv.ent
/mnt/HD1/pdb_data/1f2q/pdb/pdb1f2q.ent
/mnt/HD1/pdb_data/1f45/pdb/pdb1f45.ent
/mnt/HD1/pdb_data/1f8u/pdb/pdb1f8u.ent
/mnt/HD1/pdb_data/1fl7/pd

/mnt/HD1/pdb_data/2bc4/pdb/pdb2bc4.ent
/mnt/HD1/pdb_data/2bc4/pdb/pdb2bc4.ent
/mnt/HD1/pdb_data/2d7i/pdb/pdb2d7i.ent
/mnt/HD1/pdb_data/2d9q/pdb/pdb2d9q.ent
/mnt/HD1/pdb_data/2d9q/pdb/pdb2d9q.ent
/mnt/HD1/pdb_data/2df3/pdb/pdb2df3.ent
Desired structure doesn't exists
bad chain
/mnt/HD1/pdb_data/2e+56/pdb/pdb2e+56.ent
/mnt/HD1/pdb_data/2e9w/pdb/pdb2e9w.ent
/mnt/HD1/pdb_data/2ec8/pdb/pdb2ec8.ent
/mnt/HD1/pdb_data/2erj/pdb/pdb2erj.ent
/mnt/HD1/pdb_data/2erj/pdb/pdb2erj.ent
/mnt/HD1/pdb_data/2erj/pdb/pdb2erj.ent
/mnt/HD1/pdb_data/2f83/pdb/pdb2f83.ent
/mnt/HD1/pdb_data/2f9n/pdb/pdb2f9n.ent
/mnt/HD1/pdb_data/2gd4/pdb/pdb2gd4.ent
/mnt/HD1/pdb_data/2gd4/pdb/pdb2gd4.ent
/mnt/HD1/pdb_data/2gd4/pdb/pdb2gd4.ent
/mnt/HD1/pdb_data/2gh0/pdb/pdb2gh0.ent
/mnt/HD1/pdb_data/2gh0/pdb/pdb2gh0.ent
/mnt/HD1/pdb_data/2gj7/pdb/pdb2gj7.ent
/mnt/HD1/pdb_data/2gjx/pdb/pdb2gjx.ent
/mnt/HD1/pdb_data/2gy5/pdb/pdb2gy5.ent
/mnt/HD1/pdb_data/2gy7/pdb/pdb2gy7.ent
/mnt/HD1/pdb_data/2hev/pdb/pdb2hev.ent
/mnt/HD1/pdb_data/2

/mnt/HD1/pdb_data/3kls/pdb/pdb3kls.ent
/mnt/HD1/pdb_data/3kq4/pdb/pdb3kq4.ent
/mnt/HD1/pdb_data/3l4t/pdb/pdb3l4t.ent
/mnt/HD1/pdb_data/3l5h/pdb/pdb3l5h.ent
/mnt/HD1/pdb_data/3lb6/pdb/pdb3lb6.ent
/mnt/HD1/pdb_data/3lb6/pdb/pdb3lb6.ent
/mnt/HD1/pdb_data/3lgd/pdb/pdb3lgd.ent
/mnt/HD1/pdb_data/3lmy/pdb/pdb3lmy.ent
/mnt/HD1/pdb_data/3lqz/pdb/pdb3lqz.ent
/mnt/HD1/pdb_data/3lu9/pdb/pdb3lu9.ent
/mnt/HD1/pdb_data/3m7p/pdb/pdb3m7p.ent
/mnt/HD1/pdb_data/3mjg/pdb/pdb3mjg.ent
/mnt/HD1/pdb_data/3mjg/pdb/pdb3mjg.ent
/mnt/HD1/pdb_data/3mx0/pdb/pdb3mx0.ent
/mnt/HD1/pdb_data/3mzw/pdb/pdb3mzw.ent
/mnt/HD1/pdb_data/3n2z/pdb/pdb3n2z.ent
/mnt/HD1/pdb_data/3nvn/pdb/pdb3nvn.ent
/mnt/HD1/pdb_data/3nvn/pdb/pdb3nvn.ent
/mnt/HD1/pdb_data/3nvq/pdb/pdb3nvq.ent
/mnt/HD1/pdb_data/3nxq/pdb/pdb3nxq.ent
/mnt/HD1/pdb_data/3o4o/pdb/pdb3o4o.ent
/mnt/HD1/pdb_data/3o4o/pdb/pdb3o4o.ent
/mnt/HD1/pdb_data/3o8e/pdb/pdb3o8e.ent
/mnt/HD1/pdb_data/3o8e/pdb/pdb3o8e.ent
/mnt/HD1/pdb_data/3of6/pdb/pdb3of6.ent
/mnt/HD1/pdb_data/3of6/pd

/mnt/HD1/pdb_data/4kjy/pdb/pdb4kjy.ent
/mnt/HD1/pdb_data/4kjy/pdb/pdb4kjy.ent
/mnt/HD1/pdb_data/4kki/pdb/pdb4kki.ent
/mnt/HD1/pdb_data/4kr0/pdb/pdb4kr0.ent
/mnt/HD1/pdb_data/4kro/pdb/pdb4kro.ent
/mnt/HD1/pdb_data/4kx7/pdb/pdb4kx7.ent
/mnt/HD1/pdb_data/4kzn/pdb/pdb4kzn.ent
/mnt/HD1/pdb_data/4l0p/pdb/pdb4l0p.ent
/mnt/HD1/pdb_data/4lhu/pdb/pdb4lhu.ent
/mnt/HD1/pdb_data/4lhu/pdb/pdb4lhu.ent
/mnt/HD1/pdb_data/4lor/pdb/pdb4lor.ent
/mnt/HD1/pdb_data/4lor/pdb/pdb4lor.ent
/mnt/HD1/pdb_data/4lrh/pdb/pdb4lrh.ent
/mnt/HD1/pdb_data/4m4r/pdb/pdb4m4r.ent
/mnt/HD1/pdb_data/4mcp/pdb/pdb4mcp.ent
/mnt/HD1/pdb_data/4mcy/pdb/pdb4mcy.ent
/mnt/HD1/pdb_data/4mcz/pdb/pdb4mcz.ent
/mnt/HD1/pdb_data/4md4/pdb/pdb4md4.ent
/mnt/HD1/pdb_data/4mhx/pdb/pdb4mhx.ent
/mnt/HD1/pdb_data/4mj2/pdb/pdb4mj2.ent
/mnt/HD1/pdb_data/4mlf/pdb/pdb4mlf.ent
/mnt/HD1/pdb_data/4mqw/pdb/pdb4mqw.ent
/mnt/HD1/pdb_data/4mqw/pdb/pdb4mqw.ent
/mnt/HD1/pdb_data/4msl/pdb/pdb4msl.ent
/mnt/HD1/pdb_data/4mws/pdb/pdb4mws.ent
/mnt/HD1/pdb_data/4n0u/pd

In [32]:
def PSSM_gen(pdbID,Chain):
    gen = PSSM(work_dir='/home/yuan/Desktop/HD1/pdb_data/' + pdbID)
    gen.configure(blast_exe = '/usr/local/bin/psiblast',database='/mnt/HD1/download/uniref/uniref50.fasta',num_threads=7)
    gen.get_fasta(pdb_dir='pdb', chain=[Chain], out_dir='fasta')
    gen.get_pssm(fasta_dir='fasta', out_dir='pssm_raw', run=True)
    gen.map_pssm(pssm_dir='pssm_raw', pdb_dir='pdb', out_dir='pssm', chain=[Chain])
    gen.get_mapped_pdb(pdbpssm_dir='pssm', pdb_dir='pdb', pdbnonmatch_dir='pdb_nonmatch')

In [33]:
for index,oneRow in cd_hit_7.iterrows():
    PDB = PDBList()
    pdbID = oneRow["PDB"]
    chainOrder = oneRow["CHAIN"]
    try:
        PSSM_gen(pdbID,chainOrder)
        print(pdbID,chainOrder,"Done")
    except:
        print(pdbID,chainOrder,"Fail")


1a6a C Done

1a7c A Done

1a7s A Done

1apy C Done

1apy D Done

1au1 A Done

1auk A Done

1b0f A Done

1b41 B Done

1bhg B Done

1bik A Done

1bmo B Done

1bsi A Done

1by2 A Done

1c1z A Done

1ccz A Done
1de4 A Done

1dfv A Done
1dlh F Fail

1dmt A Done

1dr9 A Done

1elv A Done

1f2q A Done

1f45 B Done

1f8u A Done

1fl7 D Done

1fq3 A Done

1fsu A Done

1fuj C Done

1fyt D Done

1fzd D Done

1g82 D Done

1gh7 A Done

1hcn B Done

1hlg A Done

1hnf A Done

1icf J Done

1igr A Done

1imv A Done

1isf B Done
1itq A Fail

1ivo C Done

1ivy B Done

1j9c H Done

1j9c T Done

1jdp H Done

1jmj B Done

1jpy A Done

1l6x B Done

1l8j A Done

1lcf A Done
1lqv C Fail

1ly2 A Done

1m6b B Done



1md7 A Done

1mox D Done

1mx1 C Done

1n26 A Done

1n7d A Done

1nd5 D Done

1nn6 A Done

1nwr A Done

1o0v B Done

1onq C Done

1ook G Done

1ozn A Done

1p49 A Done

1ppf I Done

1pu4 B Done

1qfk L Done

1r42 A Done
1r42 C Fail

1r54 A Done
1rpq X Fail

1s78 A Done
>pdb1spj.pdb_A:
IVGGWECEQHSQPWQAALYHFSTFQCGGILVHRQWVLTAAHCISDNYQLWLGRHNLFDDENTAQFVHVSESFPHPGFNMSLLENRQADEDYSHDLMLLRLTEPADTITDAVKVVELPTEEPEVGSTCLASGWGSIEPENFSFPDDLQCVDLKILPNDECKKAHVQKVTDFMLCVGHLEGGKDTCVGDSGGPLMCDGVLQGVTSWGYVPCGTPNKPSVAVRVLSYVKWIEDTIAENSXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
>pdb1spj.A.pssm:
IVGGWECEQHSQPWQAALYHFSTFQCGGILVHRQWVLTAAHCISDNYQLWLGRHNLFDDENTAQFVHVSESFPHPGFNMSL--ERQADEDYSHDLMLLRLT


2ach A Done

2ach B Done

2ahx A Done

2ajf E Done

2aw2 X Done
2b5i A Fail

2bc4 A Done

2bc4 B Done

2d7i A Done

2d9q A Done

2d9q B Done

2df3 A Done
2e+56 A Fail
2e9w D Done

2ec8 A Done
2erj A Done

2erj B Done

2erj C Done

2f83 A Done

2f9n B Done

2gd4 B Done

2gd4 I Done

2gd4 L Done

2gh0 B Done

2gh0 C Done
2gj7 F Done

2gjx E Done

2gy5 A Done

2gy7 A Done

2hev F Done

2hev R Done

2i07 A Done

2i6s A Done

2i9b B Done
2ice T Done

2ifg B Done
2ifg E Done

2nqd A Done

2nqd B Done

2oay A Done

2obd A Done

2ok5 A Done

2oxe A Done

2p26 A Done

2pcu A Done

2pe4 A Done

2pm8 A Done

2pms D Done

2pmv D Done

2psx A Done

2qej B Done
2qki H Fail

2qqk A Done

2qqm A Done

2qt9 A Done

2qtw B Done

2seb D Done
2seb E Fail

2x2u A Done



2z7f I Done

2z80 B Done

2zck H Done

2zck P Done

3b2d A Done

3b2d C Done

3b4v C Done

3b4v F Done

3b9f I Done

3be8 B Done

3bkk A Done

3bn3 A Done

3bpl B Done
3bt1 B Done
3c5j C Fail

3cfw A Done

3cu7 B Done

3d5o D Done

3d66 B Done

3dem A Done

3di3 A Done

3di3 B Done

3e0g A Done

3ee6 A Done
3ejh E Fail

3es6 B Done
3f6k N Fail

3f8u C Done

3f8u D Done

3fcu E Done

3g04 C Done

3g5c A Done

3g6z B Done
3ghg A Done

3ghg B Done

3ghg I Done

3ghn A Done
3gxe E Fail

3h53 B Done

3h5c A Done

3h5c B Done

3hg3 B Done

3huj A Done

3iai B Done

3icu A Done

3inb B Done
3j8f 1 Done
3j8f 3 Done

3j8f 4 Done

3j8f 7 Done

3k6s C Done

3k6s F Done

3kas B Done

3kbh G Done

3kcg H Done

3kcg L Done
3kls X Done

3kq4 D Done

3l4t A Done



3l5h A Done
3lb6 A Fail
3lb6 D Fail

3lgd A Done

3lmy B Done

3lqz B Done
3lu9 F Fail

3m7p A Done

3mjg B Done

3mjg Y Done

3mx0 A Done
3mzw B Done

3n2z B Done

3nvn A Done

3nvn B Done

3nvq A Done

3nxq B Done
3o4o A Done

3o4o C Done

3o8e B Done

3o8e C Done

3of6 C Done

3of6 D Done

3ol2 A Done

3ol2 B Done

3osk A Done

3p3y A Done
3p6z I Fail

3pdf A Done

3prx D Done

3qb7 A Done

3qd6 D Done
3qd6 S Done

3qnf B Done
3qs7 C Done

3qs7 H Done
>pdb3qum.pdb_L:
DIVMTQTAPSVFVTPGESVSISCRSSKSLLHSNGNTYLYWFLQRPGQSPQLLIYRMSNLASGVPDRFSGSGSGTDFTLRISRVEAEDVGVYYCMQHLEYPVTFGAGTKVEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNECXXXX
>pdb3qum.L.pssm:
DIVMTQTAPSVFVTPGESVSISCRSSKSL--HNGNTYLYWFLQRPGQSPQLLIYRMSNLASGVPDRFSGSGSGTDFTLRISRVEAEDVGVYYCMQHLEYPVTFGAGTKVEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC----
_______________________________^___________________________

3ulu F Done

3v8x A Done

3v8x B Done

3vcm P Done

3vi3 A Done

3vi3 B Done
3w11 A Done
3w11 B Done
3w11 C Done
3w11 D Done
3w13 F Fail

3w3n A Done

3wez D Done

3wjj A Done

3wjj C Done
3wkn O Fail

4dep E Done

4dep F Done

4ebb A Done

4ejx A Done
4ejx B Done

4ejx D Done

4en0 B Done

4f7b E Done

4fdi A Done

4fgu A Done

4fqx B Done

4fxk A Done

4fxk B Done
4fxk C Done

4fyq A Done

4g1e A Done

4g1e B Done

4g8a A Done

4gdx A Done

4gdx B Done

4grw B Done

4grw C Done

4grw G Done

4gs7 A Done

4gs7 D Done

4h1s A Done
4h25 F Fail

4hsa E Done

4hsa F Done

4j4p D Done

4j4p H Done

4js1 A Done

4k9e L Done
4ki1 F Done

4kjy C Done

4kjy D Done

4kki A Done

4kr0 B Done
4kro B Done

4kx7 A Done

4kzn A Done

4l0p B Done

4lhu D Done

4lhu G Done

4lor A Done
4lor D Fail

4lrh B Done



4m4r C Done

4mcp A Done
4mcy C Fail
4mcz C Fail
4md4 C Fail

4mhx B Done

4mj2 B Done
4mlf D Fail
4mqw D Fail
4mqw Y Fail

4msl A Done

4mws B Done
4n0u A Done
4n0u D Done

4n6o B Done

4ne9 C Done

4nt5 A Done

4nzl B Done

4nzq A Done

4o3t A Done

4o3t B Done
4o3t P Fail

4obz A Done

4obz D Done

4oga E Done

4oi9 A Done

4ono A Done

4oqt A Done
4oqt H Done

4p5m C Done

4qzv D Done

4ra0 A Done

4ra0 D Done

4rsu E Done

5e8e B Done

5io1 A Done


In [34]:
count = 0
for index,oneRow in cd_hit_7.iterrows():
    PDB = PDBList()
    pdbID = oneRow["PDB"]
    chainOrder = oneRow["CHAIN"]
    if not os.path.exists("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+chainOrder+".pdb.pssm"):
        count = count + 1
count 

33

In [35]:
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
for index,oneRow in cd_hit_7.iterrows():
    PDB = PDBList()
    pdbID = oneRow["PDB"]
    chainOrder = oneRow["CHAIN"]
    if os.path.exists("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+chainOrder+".pdb.pssm"):
        with conn as cursor:
            with cursor:
                try:
                    pssm_mat = pd.read_table("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+ \
                                             chainOrder+".pdb.pssm", \
                                            comment="#",header= 0,delim_whitespace= True)
                    for row in pssm_mat.iterrows():
                        current_row = list(row)
                        query = "Insert ignore into PSSM_uniref50(PDBID,Chain,Seq,Res_type,A,R,N,D,C,Q,E,G,H,I \
                                    ,L,K,M,F,P,S,T,W,Y,V,IC) Values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, \
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                        value = [pdbID,chainOrder] + list(current_row[1])[0:2] + list(current_row[1])[4:]
                        cursor.execute(query,value)
                except Exception as e:
                    print("Exeception occured:{}".format(e))
                    print(pdbID,chainOrder)
    conn.commit()
    


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1b0f-A-62' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1b0f-A-65' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1b0f-A-99' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1b0f-A-186' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1b0f-A-188' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (106

/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1j9c-H-60' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1j9c-H-129' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1j9c-H-170' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1j9c-H-184' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '1j9c-H-188' for key 'PRIMARY'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1

## Catalytic sites' PSSM

In [6]:
peptidasesList = pd.read_csv("/home/yuan/Documents/MCSA_EC3.4_peptidases.csv") 

In [7]:
unique_chain = peptidasesList[["PDB","chain/kegg compound"]].drop_duplicates()

In [8]:
unique_chain.columns = ["PDB","CHAIN"]
unique_chain = unique_chain.reset_index(drop=True)
unique_chain.head()

,PDB,CHAIN
0,1lam,A
1,1lam,C00038
2,1lam,NaN
3,1lam,C00001
4,1lam,C00037


In [ ]:
for index,oneRow in unique_chain.iterrows():
    PDB = PDBList()
    pdbID = oneRow["PDB"]
    chainOrder = oneRow["CHAIN"]
    try:
        retrieve_status = PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "/mnt/HD1/pdb_data/" + pdbID+"/pdb", file_format="pdb")
        os.rename(retrieve_status,retrieve_status[0:(len(retrieve_status)-3)]+'pdb')
    except:
        print("bad chain")
    print(retrieve_status)

In [11]:
count = 0
for index,oneRow in unique_chain.iterrows():
    pdbID = str(oneRow["PDB"])
    chainOrder = str(oneRow["CHAIN"])
    if not os.path.exists("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+chainOrder+".pdb.pssm"):
        count = count + 1
count 

275

In [34]:
for index,oneRow in unique_chain.iterrows():
    PDB = PDBList()
    pdbID = str(oneRow["PDB"])
    chainOrder = str(oneRow["CHAIN"])
    try:
        PSSM_gen(pdbID,chainOrder)
        print(pdbID,chainOrder,"Done")
    except:
        print(pdbID,chainOrder,"Fail")


1lam A Done


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pdb2sql/pdb2sqlcore.py:466: UserWarning: SQL query get an empty
  warnings.warn('SQL query get an empty')


1lam C00038 Fail
1lam nan Fail
1lam C00001 Fail
1lam C00037 Fail

1lok A Done
1lok C00038 Fail
1lok C00001 Fail
1lok nan Fail
1lok C00151 Fail

1xgm A Done
1xgm C14818 Fail
1xgm C00001 Fail
1xgm nan Fail
1xgm C00041 Fail
1xgm C00073 Fail

1b65 A Done
1b65 C00001 Fail
1b65 C00993 Fail
1b65 C00133 Fail

1ei5 A Done
1ei5 C00001 Fail
1ei5 C00993 Fail
1ei5 C00133 Fail

1azw A Done
1azw C02830 Fail
1azw C00001 Fail
1azw C16435 Fail
1azw C05167 Fail

1a16 A Done
1a16 nan Fail
1a16 C00001 Fail
1a16 C00041 Fail
1a16 C00148 Fail

1itq A Done
1itq C00038 Fail
1itq nan Fail
1itq C00001 Fail
1itq C00151 Fail

1fy2 A Done
1fy2 C00001 Fail
1fy2 nan Fail
1fy2 C00049 Fail
1fy2 C00037 Fail

1r44 A Done
1r44 C00038 Fail
1r44 C00001 Fail
1r44 C00993 Fail
1r44 C00133 Fail

1pfq A Done
1pfq nan Fail
1pfq C00001 Fail
1pfq C02830 Fail
1pfq C05167 Fail
1o8a C00038 Fail
1o8a C00001 Fail
1o8a nan Fail
1o8a C00151 Fail

1whs A Done

1whs B Done
1whs C00001 Fail
1whs nan Fail
1whs C00151 Fail

1m4l A Done
1m4l C00

1pq5 C00001 Fail
1pq5 nan Fail
1pq5 C00151 Fail

1hzf A Done
1hzf nan Fail
1hzf C00001 Fail
1hzf C00062 Fail
1hzf C00065 Fail

1s01 A Done
1s01 nan Fail
1s01 C00001 Fail
1s01 C00151 Fail

1rtf B Done
1rtf C00001 Fail
1rtf nan Fail
1rtf C00062 Fail
1rtf C00183 Fail

1qrz A Done
1qrz C00001 Fail
1qrz nan Fail
1qrz C00062 Fail
1qrz C05167 Fail
1ds2 A Fail
1ds2 C00001 Fail
1ds2 nan Fail
1ds2 C00151 Fail

1jhf A Done
1jhf C00001 Fail
1jhf nan Fail
1jhf C01401 Fail
1jhf C00037 Fail

1t7d B Done
1t7d C00001 Fail
1t7d nan Fail
1t7d C00151 Fail

1tyf A Done
1tyf C00001 Fail
1tyf C00107 Fail
1tyf C00045 Fail

1wpo A Done
1wpo C00001 Fail
1wpo nan Fail
1wpo C00065 Fail
1wpo C00041 Fail

1rgq A Done
1rgq C00001 Fail
1rgq nan Fail
1rgq C00151 Fail

1g2i A Done

1g2i C Done
1g2i C00001 Fail
1g2i C00107 Fail
1g2i C00045 Fail

1dki A Done
1dki nan Fail
1dki C00001 Fail
1dki C00079 Fail

8pch A Done
8pch C00001 Fail
8pch nan Fail
8pch C00151 Fail

9pap A Done
9pap C00001 Fail
9pap nan Fail
9pap C00151 

1qib C00037 Fail

1kei A Done
1kei C00038 Fail
1kei C00001 Fail
1kei nan Fail
1kei C00151 Fail

1lml A Done
1lml C00038 Fail
1lml C00001 Fail
1lml nan Fail
1lml C01536 Fail
1lml C16439 Fail

1eb6 A Done
1eb6 C00038 Fail
1eb6 C00001 Fail
1eb6 nan Fail
1eb6 C00152 Fail
1eb6 C00064 Fail

1hr6 B Done
1hr6 C00038 Fail
1hr6 nan Fail
1hr6 C00001 Fail

1i1e A Done
1i1e C00038 Fail
1i1e C00001 Fail
1i1e nan Fail
1i1e C00151 Fail
1pwv A Done
1pwv C00001 Fail
1pwv nan Fail
1pwv C00151 Fail

1ryp I Done
1ryp C00001 Fail
1ryp nan Fail

1ht1 E Done
1ht1 C00001 Fail
1ht1 C00002 Fail
1ht1 nan Fail
1ht1 C00080 Fail
1ht1 C00008 Fail
1ht1 C00151 Fail
1ht1 C00009 Fail
nan nan Fail
4-dion-1(3H)-yl)-3-hydroxy-7-(phosphonooxy)hexahydro-2H-furo[3 nan Fail


In [18]:
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
for index,oneRow in unique_chain.iterrows():
    pdbID = str(oneRow["PDB"])
    chainOrder = str(oneRow["CHAIN"])
    if os.path.exists("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+chainOrder+".pdb.pssm"):
        with conn as cursor:
            with cursor:
                try:
                    
                    pssm_mat = pd.read_table("/mnt/HD1/pdb_data/"+pdbID+"/pssm/pdb"+pdbID+"."+ \
                                             chainOrder+".pdb.pssm", \
                                            comment="#",header= 0,delim_whitespace= True)
                    for row in pssm_mat.iterrows():
                        current_row = list(row)
                        query = "Insert into PSSM_uniref50(PDBID,Chain,Seq,Res_type,A,R,N,D,C,Q,E,G,H,I \
                                    ,L,K,M,F,P,S,T,W,Y,V,IC) Values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, \
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                        value = [pdbID,chainOrder] + list(current_row[1])[0:2] + list(current_row[1])[4:]
                        cursor.execute(query,value)
                        conn.commit()        
                except Exception as e:
                    print("Exeception occured:{}".format(e))
                    print(pdbID,chainOrder)
    
    
    

Exeception occured:(1062, "Duplicate entry '1lam-A-1' for key 'PRIMARY'")
1lam A
Exeception occured:(1062, "Duplicate entry '1whs-A-23' for key 'PRIMARY'")
1whs A
Exeception occured:(1062, "Duplicate entry '1whs-B-303' for key 'PRIMARY'")
1whs B
Exeception occured:(1062, "Duplicate entry '1deu-A-6' for key 'PRIMARY'")
1deu A
Exeception occured:(1062, "Duplicate entry '1ssx-A-15' for key 'PRIMARY'")
1ssx A
Exeception occured:(1062, "Duplicate entry '1rtf-B-37' for key 'PRIMARY'")
1rtf B
Exeception occured:(1062, "Duplicate entry '1dki-A-37' for key 'PRIMARY'")
1dki A
Exeception occured:(1062, "Duplicate entry '8pch-A-10' for key 'PRIMARY'")
8pch A
Exeception occured:(1062, "Duplicate entry '1qib-A-190' for key 'PRIMARY'")
1qib A
